In [1]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)
import re
import ast
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")

## reading the data

In [64]:
df_new = pd.read_excel('Tier A.xlsx')

## selecting some columns

In [153]:
a=['countyId','zipcode', 'zestimate','lotAreaValue',
       'rentZestimate','bedrooms', 'bathrooms', 'livingAreaValue', 
       'lastSoldPrice', 'zestimateLowPercent', 'zestimateHighPercent',
       'restimateLowPercent', 'restimateHighPercent', 
       'taxAssessedValue', 'taxAssessedYear', 
       'propertyTaxRate',  
       'resoFacts_hasFireplace', 'resoFacts_parking',
       'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate', 
    'city', 'county' , 'description','parentRegion_name',
    'dateSoldString', 'resoFacts_daysOnZillow', 'resoFacts_flooring', 'resoFacts_hasAttachedProperty',
    'resoFacts_hasGarage', 'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa' , 'resoFacts_hasView','resoFacts_hoaFee',
     'resoFacts_pricePerSquareFoot','resoFacts_yearBuilt' ,'solarPotential_sunScore','schools'
   ]

In [154]:
df=df_new[a]

In [155]:
df.shape

(742, 38)

In [156]:
df.isnull().sum()

countyId                                0
zipcode                                 0
zestimate                             169
lotAreaValue                           58
rentZestimate                         122
bedrooms                               22
bathrooms                               8
livingAreaValue                         9
lastSoldPrice                         222
zestimateLowPercent                   169
zestimateHighPercent                  169
restimateLowPercent                   177
restimateHighPercent                  177
taxAssessedValue                      134
taxAssessedYear                       134
propertyTaxRate                         2
resoFacts_hasFireplace                117
resoFacts_parking                       0
mortgageRates_thirtyYearFixedRate       6
mortgageRates_fifteenYearFixedRate     84
mortgageRates_arm5Rate                 84
city                                    0
county                                  0
description                       

In [157]:
df = df.dropna()

In [158]:
df.shape

(101, 38)

# Feature Engineering- Creating 6 features out of school

In [159]:
schools= list(df['schools'])

In [160]:
list_schools= [[]] * len(schools)

In [161]:
len(list_schools)

101

In [162]:
i=0
for x in schools:
    list_schools[i]=[0,0,0,0,0,0]
    x = str(x).replace('\\','"')
    l=re.findall("[{][\\a-zA-z0-9:',\-/ \.=?&]+[}]", x)
    for y in l:
        y=ast.literal_eval(y)
        if y['level']== 'Elementary':
            if y['rating']!=None:
                if list_schools[i][1]<y['rating']:
                    list_schools[i][1]=y['rating']
                    list_schools[i][0]=y['distance']
                
        if y['level']== 'Middle':
            if y['rating']!=None:
                if list_schools[i][3]<y['rating']:
                    list_schools[i][3]=y['rating']
                    list_schools[i][2]=y['distance']
        if y['level']=='High':
            if y['rating']!=None:
                if list_schools[i][5]<y['rating']:
                    list_schools[i][5]=y['rating']
                    list_schools[i][4]=y['distance']
    
    i+=1
    
    

In [163]:
list_schools[0]

[0.3, 7, 1.5, 6, 0.9, 8]

In [164]:
for i in range(len(list_schools)):
    list_schools[i] = [None if item <=0 else item for item in list_schools[i]]

In [165]:
df_3 = pd.DataFrame(list_schools, columns = ['Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance', 'Middle_Ratings', 'High_Distance', 'High_Ratings'])

In [166]:
df_3.head()

,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings
0,0.30000,7.00000,1.50000,6.00000,0.90000,8.00000
1,0.20000,6.00000,1.00000,6.00000,0.60000,8.00000
2,0.10000,8.00000,0.80000,9.00000,1.30000,7.00000
3,0.60000,7.00000,0.60000,6.00000,0.60000,7.00000
4,0.20000,4.00000,1.50000,4.00000,0.70000,3.00000


In [167]:
df_3['Elementary_Distance']=np.where(np.isnan(df_3['Elementary_Distance']),df_3['Elementary_Distance'].median(),df_3['Elementary_Distance']) 
df_3['Middle_Ratings']=np.where(np.isnan(df_3['Middle_Ratings']),df_3['Middle_Ratings'].mode(),df_3['Middle_Ratings']) 
df_3['Middle_Distance']=np.where(np.isnan(df_3['Middle_Distance']),df_3['Middle_Distance'].median(),df_3['Middle_Distance']) 
df_3['High_Ratings']=np.where(np.isnan(df_3['High_Ratings']),df_3['High_Ratings'].mode(),df_3['High_Ratings']) 
df_3['High_Distance']=np.where(np.isnan(df_3['High_Distance']),df_3['High_Distance'].median(),df_3['High_Distance']) 


In [168]:
from sklearn.impute import KNNImputer   # missing values imputation

imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_3.iloc[:,0:]=imputer.fit_transform(df_3.iloc[:,0:]) #replace df with imputed missing values

In [169]:
df_3.head()

,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings
0,0.30000,7.00000,1.50000,6.00000,0.90000,8.00000
1,0.20000,6.00000,1.00000,6.00000,0.60000,8.00000
2,0.10000,8.00000,0.80000,9.00000,1.30000,7.00000
3,0.60000,7.00000,0.60000,6.00000,0.60000,7.00000
4,0.20000,4.00000,1.50000,4.00000,0.70000,3.00000


In [170]:
df_3.shape

(101, 6)

In [171]:
df.shape

(101, 38)

In [172]:
df.drop('schools', axis=1, inplace= True)

In [173]:
df.shape

(101, 37)

In [174]:
df=df.reset_index()

In [175]:
df=pd.concat([df,df_3], axis=1)

In [176]:
df.shape  # this df contains school columns also

(101, 44)

# Feature Engineering- Creating New feature out of feature("description")

In [91]:
sentiment_des = []
from textblob import TextBlob
for text in df['description'].values:
        t= TextBlob(text)
        sentiment_des.append(t.sentiment[0])
df['desc_sentiment']=sentiment_des

In [92]:
df['desc_sentiment']=sentiment_des

In [93]:
df.drop(['description'], axis=1,inplace=True)

In [94]:
df.shape 

(426, 33)

In [95]:
df.corr()

,index,countyId,zipcode,zestimate,rentZestimate,bedrooms,bathrooms,livingAreaValue,propertyTaxRate,resoFacts_hasFireplace,resoFacts_parking,mortgageRates_thirtyYearFixedRate,mortgageRates_fifteenYearFixedRate,mortgageRates_arm5Rate,resoFacts_daysOnZillow,resoFacts_hasAttachedProperty,resoFacts_hasGarage,resoFacts_hasPetsAllowed,resoFacts_hasSpa,resoFacts_hasView,resoFacts_pricePerSquareFoot,resoFacts_yearBuilt,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings,desc_sentiment
index,1.00000,-0.50481,0.54942,-0.01205,-0.19526,0.31321,0.18026,0.10210,0.58811,0.32145,0.09001,0.12736,0.10649,-0.04034,0.27657,-0.52613,-0.15861,-0.26418,0.07075,-0.15623,-0.18731,0.13588,0.06345,0.14162,0.02158,0.29266,0.33295,0.34586,-0.01819
countyId,-0.50481,1.00000,-0.62654,0.07718,0.15793,-0.37873,-0.31020,-0.23746,-0.65182,-0.23948,-0.43720,-0.03456,0.06579,-0.12180,-0.31275,0.42449,-0.29703,0.15246,-0.03150,0.04042,0.47200,-0.13285,-0.19843,-0.19789,-0.01288,-0.41652,-0.30578,-0.45044,-0.00046
zipcode,0.54942,-0.62654,1.00000,-0.14505,-0.21460,0.26834,0.17214,0.11642,0.55402,0.26118,0.23794,0.02043,-0.02746,0.09214,0.17017,-0.42250,0.05377,-0.18831,0.08112,-0.10515,-0.39657,0.12664,0.11857,0.04046,0.07265,0.25879,0.32567,0.29669,0.03919
zestimate,-0.01205,0.07718,-0.14505,1.00000,0.88699,0.47670,0.62408,0.77415,-0.10598,0.17093,0.26846,-0.07897,0.47192,-0.46992,0.30876,-0.02162,0.00237,0.02945,-0.06601,0.20250,0.42750,-0.08912,0.08651,0.18616,-0.03737,0.14231,0.00140,0.06138,0.06807
rentZestimate,-0.19526,0.15793,-0.21460,0.88699,1.00000,0.37840,0.58132,0.71840,-0.24709,0.04888,0.26107,-0.05402,0.36071,-0.37385,0.24459,0.10797,0.03701,0.07151,-0.06199,0.22209,0.31277,-0.17201,0.05785,0.08745,-0.03996,0.04903,-0.08602,-0.04862,0.04214
bedrooms,0.31321,-0.37873,0.26834,0.47670,0.37840,1.00000,0.69469,0.70097,0.31699,0.39983,0.31849,-0.00633,0.48092,-0.45375,0.32869,-0.35843,0.08521,-0.28310,0.03861,-0.00481,-0.22319,0.05885,0.11335,0.19649,-0.07926,0.28525,0.18448,0.32931,0.03737
bathrooms,0.18026,-0.31020,0.17214,0.62408,0.58132,0.69469,1.00000,0.81834,0.19206,0.23944,0.40806,-0.01047,0.37214,-0.34579,0.40326,-0.22257,0.14402,-0.08366,-0.00633,0.14499,-0.17581,0.02540,0.14825,0.18577,0.04357,0.24610,0.21375,0.27722,0.00021
livingAreaValue,0.10210,-0.23746,0.11642,0.77415,0.71840,0.70097,0.81834,1.00000,0.11437,0.25940,0.45364,-0.01635,0.38251,-0.36783,0.51606,-0.18155,0.16519,-0.02798,0.00515,0.17028,-0.12525,0.01269,0.20491,0.18039,0.05057,0.24305,0.14712,0.19142,0.03019
propertyTaxRate,0.58811,-0.65182,0.55402,-0.10598,-0.24709,0.31699,0.19206,0.11437,1.00000,0.32145,0.19546,0.07331,0.01541,0.03331,0.24066,-0.48631,-0.02403,-0.29850,0.09764,-0.12767,-0.31505,0.24425,0.18703,0.12269,-0.01258,0.28996,0.36637,0.34799,0.06083
resoFacts_hasFireplace,0.32145,-0.23948,0.26118,0.17093,0.04888,0.39983,0.23944,0.25940,0.32145,1.00000,0.11929,0.05996,0.17678,-0.16240,0.09794,-0.32744,-0.02165,-0.26903,0.15789,-0.03410,-0.16151,0.05537,0.04989,0.12363,-0.14405,0.30719,0.13281,0.37174,0.13992


# Train-Test Split

In [96]:
df.select_dtypes(include='number').columns

Index(['index', 'countyId', 'zipcode', 'zestimate', 'rentZestimate',
       'bedrooms', 'bathrooms', 'livingAreaValue', 'propertyTaxRate',
       'resoFacts_hasFireplace', 'resoFacts_parking',
       'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate',
       'resoFacts_daysOnZillow', 'resoFacts_hasAttachedProperty',
       'resoFacts_hasGarage', 'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa',
       'resoFacts_hasView', 'resoFacts_pricePerSquareFoot',
       'resoFacts_yearBuilt', 'Elementary_Distance', 'Elementary_Ratings',
       'Middle_Distance', 'Middle_Ratings', 'High_Distance', 'High_Ratings',
       'desc_sentiment'],
      dtype='object')

In [97]:
y = df.zestimate

In [98]:
x= df.drop('zestimate', axis=1)

In [99]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2, random_state=123)

In [100]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(340, 32)
(340,)
(86, 32)
(86,)


# One Hot Encoding

In [101]:
cat_columns=['city', 'county']

In [105]:
from scipy.sparse import coo_matrix

X_train_cat_processed = coo_matrix((340, 0))
X_test_cat_processed = coo_matrix((86, 0))

In [106]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

from sklearn import preprocessing
vectorizer= preprocessing.OneHotEncoder(handle_unknown = 'ignore')
for x in cat_columns:
    
    
# 1. INSTANTIATE
    
    

        
    one_hot_train = vectorizer.fit_transform(X_train[x].values.reshape(-1,1))
    one_hot_test = vectorizer.transform(X_test[x].values.reshape(-1,1))
    vectorizer.get_feature_names()
    
    X_train_cat_processed= hstack((X_train_cat_processed,one_hot_train)).tocsr()
    X_test_cat_processed= hstack((X_test_cat_processed,one_hot_test)).tocsr()



In [107]:
print(X_train_cat_processed.shape)
print(X_test_cat_processed.shape)

(340, 14)
(86, 14)


# Numeric Data Preparation

In [140]:
from scipy.sparse import coo_matrix

X_train_num_processed = coo_matrix((340,0))
X_test_num_processed = coo_matrix((86,0))

In [141]:
num_cols= [ 
       'bedrooms', 'livingAreaValue', 
        'resoFacts_hasFireplace',
       'resoFacts_hasGarage',
       'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa', 'resoFacts_hasView',
       'resoFacts_pricePerSquareFoot', 
       ]

In [142]:
from scipy.sparse import hstack

for x in num_cols:
    

    s_train = (X_train[x].values.reshape(-1, 1))

    s_test = (X_test[x].values.reshape(-1, 1))

    
    X_train_num_processed= hstack((X_train_num_processed,s_train)).tocsr()
    X_test_num_processed= hstack((X_test_num_processed,s_test)).tocsr()


In [143]:
print(X_train_num_processed.shape)
print(X_test_num_processed.shape)

(340, 8)
(86, 8)


# Linear Regression

In [144]:
from scipy.sparse import hstack
X_train_processed = hstack((X_train_cat_processed,X_train_num_processed )).tocsr()
X_test_processed = hstack(( X_test_cat_processed,X_test_num_processed)).tocsr()
print(X_test_processed.shape)
print(X_train_processed.shape)

(86, 22)
(340, 22)


In [145]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train_processed, Y_train)

LinearRegression()

In [146]:
Y_pred = regressor.predict(X_test_processed)

In [147]:
df_out = pd.DataFrame({'Actual': Y_test, 'Predicted': Y_pred})
df_out.head(8)

,Actual,Predicted
13,2320100.00000,2314103.64795
121,317200.00000,-160766.86229
351,1475132.00000,1801897.32820
279,1065800.00000,893070.62218
173,1852800.00000,1818940.00229
288,960064.00000,1343734.15352
120,969500.00000,1665460.94071
15,2599100.00000,2792026.35512


In [148]:
from sklearn.metrics import r2_score
r2_score =r2_score(Y_test, Y_pred)
rows , col =X_train_processed.shape
adj_R2 = 1- ((1-r2_score) * (rows-1)/(rows-col-1)) #Adj R2 = 1-(1-R2)*(n-1)/(n-p-1)
print('R2 is:', r2_score.round(4))
print('Adjusted R2 is:', adj_R2.round(3))


R2 is: 0.8741
Adjusted R2 is: 0.865


# OLS regresssion

In [151]:
X_train_test_processed=pd.concat([pd.DataFrame(X_train_num_processed.todense()),pd.DataFrame(X_test_num_processed.todense())],ignore_index=True)
Y_train_test_processed = pd.concat([pd.DataFrame(Y_train),pd.DataFrame(Y_test)], ignore_index=True)

In [152]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


X2 = sm.add_constant(X_train_test_processed)
est = sm.OLS(Y_train_test_processed, X2)
est2 = est.fit()
print(est2.summary())



                            OLS Regression Results                            
Dep. Variable:              zestimate   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     386.7
Date:                Thu, 15 Jun 2023   Prob (F-statistic):          1.31e-187
Time:                        18:33:47   Log-Likelihood:                -6086.4
No. Observations:                 426   AIC:                         1.219e+04
Df Residuals:                     417   BIC:                         1.223e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.537e+06   9.59e+04    -16.029      0.0

# Interpretation

1.const (Intercept): The constant term is -2.43e+05 or -$243,000. This can be interpreted as the baseline value of `zestimate` when all the predictor variables are zero. However, interpreting the intercept might not make sense in this context since it's not meaningful to have 0 bedrooms, 0 rentZestimate, etc.

2.bedrooms (0): The coefficient for bedrooms is 1.046e+05 or $104,600. This means that, on average, for each additional bedroom, the `zestimate` is expected to increase by approximately $104,600, holding other variables constant. The p-value is close to 0, indicating that the number of bedrooms is a statistically significant predictor of the `zestimate`.

3. rentZestimate (1): The coefficient for rentZestimate is 287.4495, or approximately $287.45. This means that, on average, for each additional dollar in the rentZestimate, the `zestimate` is expected to increase by approximately $287.45, holding other variables constant. The p-value is close to 0, which indicates that rentZestimate is a statistically significant predictor.

4.lastSoldPrice (2): The coefficient for lastSoldPrice is 0.1722. This implies that for each additional dollar in the lastSoldPrice, the `zestimate` is expected to increase by $0.1722, holding other variables constant. The p-value is also close to 0, indicating that lastSoldPrice is a statistically significant predictor.

5.*resoFacts_hasFireplace (3): The coefficient for resoFacts_hasFireplace is 1.269e+05 or $126,900. Since resoFacts_hasFireplace is likely a binary variable (indicating the presence or absence of a fireplace), this coefficient can be interpreted as the average change in `zestimate` when a property has a fireplace compared to not having one, holding other variables constant. The p-value is small (0.002), indicating that having a fireplace is a statistically significant predictor of the `zestimate`.
